<a href="https://colab.research.google.com/github/Sanjanathakal/BIA-678/blob/master/ProjectBIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Uploading data from the drive :

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Importing Required Libraries: 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dot, Add, Flatten
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import SGD, Adam, Adamax


Importing training and testing csv : 

In [ ]:
Train_data=pd.read_csv("/content/drive/MyDrive/BIA PROJECT /DATA/newTraindata.csv")
Test_data = pd.read_csv("/content/drive/MyDrive/BIA PROJECT /DATA/newtestdata.csv")

Train_data.head()
Test_data.head()

,Unnamed: 0,customer_id,id,vendor_rating
0,0,ICE2DJP,4.0,4.4
1,1,ICE2DJP,13.0,4.7
2,2,ICE2DJP,20.0,4.5
3,3,ICE2DJP,23.0,4.5
4,4,ICE2DJP,28.0,4.4


In [ ]:
Train_data.head()

,Unnamed: 0,customer_id,id,vendor_rating
0,0,TCHWPBT,4,4.4
1,1,TCHWPBT,13,4.7
2,2,TCHWPBT,20,4.5
3,3,TCHWPBT,23,4.5
4,4,TCHWPBT,28,4.4


In [ ]:
Train_data[['customer_id','id','vendor_rating']]

,customer_id,id,vendor_rating
0,TCHWPBT,4,4.4
1,TCHWPBT,13,4.7
2,TCHWPBT,20,4.5
3,TCHWPBT,23,4.5
4,TCHWPBT,28,4.4
...,...,...,...
5802395,LBMRK7A,849,4.1
5802396,LBMRK7A,855,4.2
5802397,LBMRK7A,856,4.3
5802398,LBMRK7A,858,4.2


In [ ]:
Test_data[['customer_id','id','vendor_rating']]

,customer_id,id,vendor_rating
0,ICE2DJP,4.0,4.4
1,ICE2DJP,13.0,4.7
2,ICE2DJP,20.0,4.5
3,ICE2DJP,23.0,4.5
4,ICE2DJP,28.0,4.4
...,...,...,...
1671995,NaN,NaN,NaN
1671996,NaN,NaN,NaN
1671997,NaN,NaN,NaN
1671998,NaN,NaN,NaN


In [ ]:
resturants = Train_data['id'].unique()
print(len(resturants))
print(resturants)

100
[  4  13  20  23  28  33  43  44  55  66  67  75  76  78  79  81  82  83
  84  85  86  90  92 104 105 106 110 113 115 134 145 148 149 154 157 159
 160 161 176 180 188 189 191 192 193 195 196 197 199 201 203 207 216 221
 225 231 237 243 250 259 265 271 274 288 289 294 295 298 299 300 303 304
 310 356 386 391 398 401 419 459 537 547 573 575 577 578 582 583 676 679
 681 841 843 845 846 849 855 856 858 907]


In [ ]:
users = Train_data['customer_id'].unique()
print(len(users))
print(users)

34523
['TCHWPBT' 'ZGFSYCZ' 'S2ALZFL' ... 'SQMJ08H' '9LW9CHN' 'LBMRK7A']


In [ ]:
data = pd.DataFrame(Train_data,columns= [['customer_id','id','vendor_rating']])

In [ ]:
import numpy as np
import pandas as pd
from scipy.linalg import sqrtm
def create_utility_matrix(data, formatizer = {'user':0, 'Resturant': 1, 'Rating': 2}):
    """
        :param data:      Array-like, 2D, nx3
        :param formatizer:pass the formatizer
        :return:          utility matrix (n x m), n=users, m=items
    """
        
    ResturantField = formatizer['Resturant']
    userField = formatizer['user']
    RatingField = formatizer['Rating']
    
    userList = data.iloc[:,userField].tolist()
    ResturantList = data.iloc[:,ResturantField].tolist()
    RatingList = data.iloc[:,RatingField].tolist()
    
    users = list(set(data.iloc[:,userField]))
    Resturants = list(set(data.iloc[:,ResturantField]))
    
    users_index = {users[i]: i for i in range(len(users))}
    
    pd_dict = {Resturant: [np.nan for i in range(len(users))] for Resturant in Resturants}
    
    for i in range(0,len(data)):
        Resturant = ResturantList[i]
        user = userList[i]
        Rating = RatingList[i]
    
    pd_dict[Resturant][users_index[user]] = Rating
    
    X = pd.DataFrame(pd_dict)
    X.index = users
        
    Resturantcols = list(X.columns)
    Resturant_index = {Resturantcols[i]: i for i in range(len(Resturantcols))}
    # users_index gives us a mapping of user_id to index of user
    # Resturants_index provides the same for Resturants
    return X, users_index, Resturant_index

In [ ]:
def svd(train, k):

    utilMat = np.array(train)
  
    # the nan or unavailable entries are masked
    mask = np.isnan(utilMat)
    masked_arr = np.ma.masked_array(utilMat, mask)
    item_means = np.mean(masked_arr, axis=0)
  
    # nan entries will replaced by the average rating for each item
    utilMat = masked_arr.filled(item_means)
    x = np.tile(item_means, (utilMat.shape[0],1))
  
    # we remove the per item average from all entries.
  
    # the above mentioned nan entries will be essentially zero now
    utilMat = utilMat - x
  
    # The magic happens here. U and V are user and item features
    U, s, V=np.linalg.svd(utilMat, full_matrices=False)
    s=np.diag(s)
  
    # we take only the k most significant features
    s=s[0:k,0:k]
    U=U[:,0:k]
    V=V[0:k,:]
  
    s_root=sqrtm(s)
  
    Usk=np.dot(U,s_root)
    skV=np.dot(s_root,V)
    UsV = np.dot(Usk, skV)
    UsV = UsV + x
  
    print("svd done")
  
    return UsV

In [ ]:
def rmse(true, pred):

    # this will be used towards the end
    x = true - pred
    return sum([xi*xi for xi in x])/len(x)

# to test the performance over a different number of features
no_of_features = [8,10,12,14,17]

utilMat, users_index, items_index = create_utility_matrix(train)

for f in no_of_features: 
    svdout = svd(utilMat, k=f)
    pred = [] #to store the predicted ratings

    for _,row in test.iterrows():
        user = row['userId']
        item = row['VendorId']

        u_index = users_index[user]
        if item in items_index:
            i_index = items_index[item]
            pred_rating = svdout[u_index, i_index]
        else:
            pred_rating = np.mean(svdout[u_index, :])
        pred.append(pred_rating)

print(rmse(test['rating'], pred))